# ch14 Working with CSV Files and JSON Data

## Working with CSV Files and JSON Data

- PDF와 Word는 Binary format 이었지만 CSV와 JSON 파일은 plain text다.

## The CSV Module

In [1]:
!cat src/example.csv

### CSV features

- Don't have types for their values - everything is a string
- Don't have settings for font size or color
- Don't have multiple worksheets
- Can't specify cell widths and heights
- Can't have merged cells
- Can't have images or charts embedded in them


### CSV advantages

- simplicity
- 많은 프로그램에서 csv 파일을 지원한다.
- split() 로만 하게 되면 잠재적인 escape character를 처리할 수 없다.
- 너는 항상 csv 모듈을 써서 읽고 써라.

### Reader Objects

In [2]:
import csv

In [3]:
with open('src/example.csv') as exampleFile:
    exampleReader = csv.reader(exampleFile)
    exampleData = list(exampleReader)

In [4]:
exampleData

[['4/5/2014 13:34', 'Apples', '73'],
 ['4/5/2014 3:41', 'Cherries', '85'],
 ['4/6/2014 12:46', 'Pears', '14'],
 ['4/8/2014 8:59', 'Oranges', '52'],
 ['4/10/2014 2:07', 'Apples', '152'],
 ['4/10/2014 18:10', 'Bananas', '23'],
 ['4/10/2014 2:40', 'Strawberries', '98']]

In [7]:
exampleReader

In [8]:
exampleReader()

TypeError: '_csv.reader' object is not callable

In [9]:
exampleData[0][0]

'4/5/2014 13:34'

In [10]:
exampleData[0][1]

'Apples'

In [12]:
exampleData[0][3]

IndexError: list index out of range

In [13]:
exampleData[1][1]

'Cherries'

In [14]:
exampleData[6][1]

'Strawberries'

### Reading Data from Reader Objects in a for Loop

In [15]:
import csv

In [25]:
with open('src/example.csv') as exampleFile:
    exampleReader = csv.reader(exampleFile)
    for row in exampleReader:
        print('Row # {} {}'.format(exampleReader.line_num, row))

Row # 1 ['4/5/2014 13:34', 'Apples', '73']
Row # 2 ['4/5/2014 3:41', 'Cherries', '85']
Row # 3 ['4/6/2014 12:46', 'Pears', '14']
Row # 4 ['4/8/2014 8:59', 'Oranges', '52']
Row # 5 ['4/10/2014 2:07', 'Apples', '152']
Row # 6 ['4/10/2014 18:10', 'Bananas', '23']
Row # 7 ['4/10/2014 2:40', 'Strawberries', '98']


### Writer Objects

- [2. Built\-in Functions — Python 2.7.10 documentation](https://docs.python.org/2/library/functions.html#open): newline

In [26]:
import csv

In [31]:
outputFile = open('output.csv', 'w')

In [32]:
outputWriter = csv.writer(outputFile)

In [33]:
outputWriter.writerow(['spam', 'eggs', 'bacon', 'ham'])

In [34]:
outputWriter.writerow(['Hello, world!', 'eggs', 'bacon', 'ham'])

In [35]:
outputWriter.writerow([1, 2, 3, 3.141592, 4])

In [36]:
outputFile.close()

In [37]:
# ,가 있어서 ""로 escape 시켜줌
!cat output.csv

### The delimiter and lineterminator Keyword Arguments

In [44]:
import csv
csvFile = open('example.tsv', 'w')
csvWriter = csv.writer(csvFile, delimiter='\t', lineterminator='\n\n')

In [45]:
csvWriter.writerow(['apples', 'oranges', 'grapes'])

In [46]:
csvWriter.writerow(['eggs', 'bacon', 'ham'])

In [47]:
csvWriter.writerow(['spam', 'spam', 'spam', 'spam', 'spam', 'spam'])

In [48]:
csvFile.close()

In [49]:
!cat example.tsv

apples	oranges	grapes

eggs	bacon	ham

spam	spam	spam	spam	spam	spam



## Project: Removing the Header from CSV Files

### High level Logic

- Find all the CSV files in the current working directory.
- Read in the full contents of each file.
- Write out the contents, skipping the first line, to a new CSV file.

### Code level Logic

- Loop over a list of files from os.listdir(), skipping the non-CSV files.
- Create a CSV reader object and read in the contents of the file, using the line_num attribute to figure out which line to skip.
- Create a CSV Writer object and write out the read-in data to the new file.

### Step 1: Loop Through Each CSV File

#### os.makedirs exist_ok

- [mkdir \-p functionality in python \- Stack Overflow](http://stackoverflow.com/questions/600268/mkdir-p-functionality-in-python): 3.2 이상만 되네

In [43]:
import platform
int(platform.python_version_tuple()[0])

2

In [35]:
os.path.dirname('/tmp/headerRemoved/')

'/tmp/headerRemoved'

In [12]:
help(os.path.dirname)

Help on function dirname in module posixpath:

dirname(p)
    Returns the directory component of a pathname



In [47]:
l = os.listdir('.')

In [50]:
l[5]

'census2010.py'

In [56]:
os.path.splitext??

In [53]:
os.path.splitext(l[5])

('census2010', '.py')

#### To get only text extension

- [Extracting extension from filename in Python \- Stack Overflow](http://stackoverflow.com/questions/541390/extracting-extension-from-filename-in-python)

In [61]:
os.path.splitext('census2010.10.py')[1]

'.py'

In [64]:
os.path.splitext('census2010.10.py')[1][1:]

'py'

In [69]:
%%writefile removeCsvHeader/removecsvheader.py
# os.makedirs test purpose
import csv
import os
import platform

def ensure_dir(dir_):
    if not os.path.exists(dir_):
        os.makedirs(dir_)

python_version = int(platform.python_version_tuple()[0])
if python_version == 3: 
    os.makedirs('headerRemoved', exist_ok=True)
else:
    ensure_dir('headerRemoved')

# Looop through every file in the current working directory.
for csvFilename in os.listdir('.'):
    if not csvFilename.endswith('.csv'):
        continue # skip non-csv files
    print('Removing header from {}...'.format(csvFilename))
    
    # TODO: Read the CSV file in (skipping first row).
    csvRows = []
    with open(csvFilename) as csvFileObj:
        readerObj = csv.reader(csvFileObj)
        for row in readerObj:
            if readerObj.line_num == 1:
                continue
            csvRows.append(row)
    
    # TODO: Write out the CSV file.
    with open(os.path.join('headerRemoved', csvFilename), 'w') as csvFileObj:
        csvWriter = csv.writer(csvFileObj)
        for row in csvRows:
            csvWriter.writerow(row)

Writing removeCsvHeader/removecsvheader.py


In [75]:
%cd removeCsvHeader/

/Users/re4lfl0w/Documents/ipython/books/Automate_the_Boring_Sutff_with_Python/removeCsvHeader


In [76]:
!python removecsvheader.py

Removing header from NAICS_data_1048.csv...
Removing header from NAICS_data_1218.csv...
Removing header from NAICS_data_1657.csv...
Removing header from NAICS_data_1751.csv...
Removing header from NAICS_data_1814.csv...
Removing header from NAICS_data_1817.csv...
Removing header from NAICS_data_1889.csv...
Removing header from NAICS_data_1952.csv...
Removing header from NAICS_data_1973.csv...
Removing header from NAICS_data_2066.csv...
Removing header from NAICS_data_2092.csv...
Removing header from NAICS_data_2183.csv...
Removing header from NAICS_data_2346.csv...
Removing header from NAICS_data_2427.csv...
Removing header from NAICS_data_2648.csv...
Removing header from NAICS_data_2799.csv...
Removing header from NAICS_data_2828.csv...
Removing header from NAICS_data_2959.csv...
Removing header from NAICS_data_2988.csv...
Removing header from NAICS_data_2993.csv...
Removing header from NAICS_data_2994.csv...
Removing header from NAICS_data_3044.csv...
Removing header from NAICS_data_

In [78]:
!ls -l headerRemoved/

total 6408
-rw-r--r--  1 re4lfl0w  staff  26615  6  7 13:11 NAICS_data_1048.csv
-rw-r--r--  1 re4lfl0w  staff  24255  6  7 13:11 NAICS_data_1218.csv
-rw-r--r--  1 re4lfl0w  staff  46908  6  7 13:11 NAICS_data_1657.csv
-rw-r--r--  1 re4lfl0w  staff  24707  6  7 13:11 NAICS_data_1751.csv
-rw-r--r--  1 re4lfl0w  staff  20775  6  7 13:11 NAICS_data_1814.csv
-rw-r--r--  1 re4lfl0w  staff  24738  6  7 13:11 NAICS_data_1817.csv
-rw-r--r--  1 re4lfl0w  staff  34045  6  7 13:11 NAICS_data_1889.csv
-rw-r--r--  1 re4lfl0w  staff  35990  6  7 13:11 NAICS_data_1952.csv
-rw-r--r--  1 re4lfl0w  staff  27629  6  7 13:11 NAICS_data_1973.csv
-rw-r--r--  1 re4lfl0w  staff  35486  6  7 13:11 NAICS_data_2066.csv
-rw-r--r--  1 re4lfl0w  staff  25764  6  7 13:11 NAICS_data_2092.csv
-rw-r--r--  1 re4lfl0w  staff  44995  6  7 13:11 NAICS_data_2183.csv
-rw-r--r--  1 re4lfl0w  staff  24911  6  7 13:11 NAICS_data_2346.csv
-rw-r--r--  1 re4lfl0w  staff  39792  6  7 13:11 NAICS_data_2427.csv
-rw-r--r--  1 re4lfl0w 

In [79]:
!head NAICS_data_1048.csv

In [80]:
!head headerRemoved/NAICS_data_1048.csv

In [81]:
%cd ..

/Users/re4lfl0w/Documents/ipython/books/Automate_the_Boring_Sutff_with_Python


#### 문제점

- 무작정 삭제하니 첫번째 줄이 헤더줄인지 아닌지 알 수가 없네
- 이걸 어떻게 구분해서 헤더가 있는 csv 파일만 헤더를 삭제하고 헤더가 없는건 그냥 놔두지?
- 헤더가 없는 csv 파일을 건드려서 원본 data가 손상됐잖아.

In [68]:
!cat output.csv

In [67]:
!cat headerRemoved/output.csv

### Step 2: Read in the CSV File

### Step 3: Write Out the CSV file Without First Row

- vice versa: 거꾸로

### Ideas for Similar Programs

- Compare data between different rows in a CSV file or between multiple CSV files.
- Copy specific data from a CSV file to an Excel file, or vice versa.
- Check for invalid data or formatting mistakes in CSV files and alert the user to these errors.
- Read data from a CSV file as input for your Python programs.

## JSON and APIs

### API advantages

- Scrape raw data from Web sites(Accessing APIs is often more convenient than downloading web pages and parsing HTML with Beautiful Soup.)
- Automatically download new posts from one of your social network accounts and post them to another account. For example, you could take your Tumblr posts and post them to Facebook.
- Create a "movie encyclopedia" for your personal movie collection by pulling data from IMDb, Rotten Tomatoes, and Wikipedia and putting it into a single text file on your computer.

## The JSON Module

- json.loads()
- json.dumps()
- JSON은 모든 파이썬 값들을 저장할 수 없다.
- 오직 strings, integers, floats, Booleans, lists, dictionaries, and NoneType만 가능하다.
- File objects, CSV Reader or Writer objects, Regex objects, or Selenium WebElement objects는 불가능하다.

In [82]:
import json

In [85]:
json.loads?

In [83]:
help(json.loads)

Help on function loads in module json:

loads(s, encoding=None, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)
    Deserialize ``s`` (a ``str`` or ``unicode`` instance containing a JSON
    document) to a Python object.
    
    If ``s`` is a ``str`` instance and is encoded with an ASCII based encoding
    other than utf-8 (e.g. latin-1) then an appropriate ``encoding`` name
    must be specified. Encodings that are not ASCII based (such as UCS-2)
    are not allowed and should be decoded to ``unicode`` first.
    
    ``object_hook`` is an optional function that will be called with the
    result of any object literal decode (a ``dict``). The return value of
    ``object_hook`` will be used instead of the ``dict``. This feature
    can be used to implement custom decoders (e.g. JSON-RPC class hinting).
    
    ``object_pairs_hook`` is an optional function that will be called with the
    result of any object literal de

In [84]:
help(json.dumps)

Help on function dumps in module json:

dumps(obj, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, encoding='utf-8', default=None, sort_keys=False, **kw)
    Serialize ``obj`` to a JSON formatted ``str``.
    
    If ``skipkeys`` is false then ``dict`` keys that are not basic types
    (``str``, ``unicode``, ``int``, ``long``, ``float``, ``bool``, ``None``)
    will be skipped instead of raising a ``TypeError``.
    
    If ``ensure_ascii`` is false, all non-ASCII characters are not escaped, and
    the return value may be a ``unicode`` instance. See ``dump`` for details.
    
    If ``check_circular`` is false, then the circular reference check
    for container types will be skipped and a circular reference will
    result in an ``OverflowError`` (or worse).
    
    If ``allow_nan`` is false, then it will be a ``ValueError`` to
    serialize out of range ``float`` values (``nan``, ``inf``, ``-inf``) in
    strict compli

### Reading JSON with the loads() Function

- loads(load string)

In [86]:
stringOfJsonData = '{"name": "Zophie", "isCat": true, "miceCaught": 0, "felineIQ": null}'

In [91]:
type(stringOfJsonData)

str

In [87]:
import json

In [88]:
jsonDataAsPythonValue = json.loads(stringOfJsonData)

In [89]:
jsonDataAsPythonValue

{u'felineIQ': None, u'isCat': True, u'miceCaught': 0, u'name': u'Zophie'}

In [90]:
type(jsonDataAsPythonValue)

dict

- JSON strings은 항상 double quotes를 사용한다.
- 돌려주는 것은 파이썬 사전과 같다.
- 딕셔너리는 순서가 없고, key-value pairs다.

### Writing JSON with the dumps() Function

In [93]:
pythonValue = {'isCat': True, 'miceCaught': 0, 'name': 'Zophie', 'felineIQ': None}

In [94]:
import json

In [95]:
stringOfJsonData = json.dumps(pythonValue)

In [96]:
stringOfJsonData

'{"miceCaught": 0, "isCat": true, "felineIQ": null, "name": "Zophie"}'

- JSON에 포함될 수 있는 데이터 타입
  - dictionary
  - list
  - integer
  - float
  - string
  - Boolean
  - None

## Project: Fetching Current Weather Data

### High Level Logic

- Reads the requested location from the command line.
- Downloads JSON weather data from OpenWeatherMap.org
- Converts the string of JSON data to a Python data structure.
- Print the weather for today and the next two days.

### Code Level Logic

- Join strings in sys.argv to get the location
- Call requests.get() to download the weather data.
- Call json.loads() to convert the JSON data to a Python data structure.
- Print the weather forecast.

### Step 1: Get Location from the Command Line Argument

In [97]:
import json
import requests
import optparse


def main():
    parser = optparse.OptionParser(usage='%prog -l <location>', version='0.1')
    parser.add_option('-l',
                      dest='location',
                      type='string',
                      help='To insert location')

    (options, args) = parser.parse_args()
    
    # TODO: Download the JSON data from OpenWeatherMap.org's API.
    
    # TODO: Load JSOn data into a Python variable.
    
if __name__ == '__main__':
    main()

### Optparse 연습

- [optparse – Command line option parser to replace getopt. \- Python Module of the Week](http://pymotw.com/2/optparse/)

In [103]:
import optparse

parser = optparse.OptionParser()
parser.add_option('-a', action="store_true", default=False)
parser.add_option('-b', action="store", dest="b")
parser.add_option('-c', action="store", dest="c", type="int")

print parser.parse_args(['-a', '-bval', '-c', '3'])

(<Values at 0x110458998: {'a': False, 'c': 3, 'b': 'val'}>, [])


In [105]:
# -a를 설정하지 않으면 기본값인 False로 세팅되는구나. 
# dest는 만들지도 않았는데 어떻게 저장되지?
import optparse

parser = optparse.OptionParser()
parser.add_option('-a', action="store_true", default=False)
parser.add_option('-b', action="store", dest="b")
parser.add_option('-c', action="store", dest="c", type="int")

print parser.parse_args(['-bval', '-c', '3'])

(<Values at 0x110458f38: {'a': False, 'c': 3, 'b': 'val'}>, [])


In [106]:
import optparse

parser = optparse.OptionParser()
parser.add_option('-a', action="store_true", default=False)
parser.add_option('-b', action="store")
parser.add_option('-c', action="store", dest="c", type="int")

print parser.parse_args(['-a', '-bval', '-c', '3'])

(<Values at 0x11045f5f0: {'a': True, 'c': 3, 'b': 'val'}>, [])


#### Callbacks

In [107]:
import optparse

def flag_callback(option, opt_str, value, parser):
    print 'flag_callback:'
    print '\toption:', repr(option)
    print '\topt_str:', opt_str
    print '\tvalue:', value
    print '\tparser:', parser
    return

def with_callback(option, opt_str, value, parser):
    print 'with_callback:'
    print '\toption:', repr(option)
    print '\topt_str:', opt_str
    print '\tvalue:', value
    print '\tparser:', parser
    return

parser = optparse.OptionParser()
parser.add_option('--flag', action="callback", callback=flag_callback)
parser.add_option('--with', 
                  action="callback",
                  callback=with_callback,
                  type="string",
                  help="Include optional feature")

parser.parse_args(['--with', 'foo', '--flag'])

with_callback:
	option: <Option at 0x110458ef0: --with>
	opt_str: --with
	value: foo
	parser: <optparse.OptionParser instance at 0x1104641b8>
flag_callback:
	option: <Option at 0x110458d40: --flag>
	opt_str: --flag
	value: None
	parser: <optparse.OptionParser instance at 0x1104641b8>


(<Values at 0x110458e60: {'with': None}>, [])

In [108]:
import optparse

def with_callback(option, opt_str, value, parser):
    print 'with_callback:'
    print '\toption:', repr(option)
    print '\topt_str:', opt_str
    print '\tvalue:', value
    print '\tparser:', parser
    return

parser = optparse.OptionParser()
parser.add_option('--with', 
                  action="callback",
                  callback=with_callback,
                  type="string",
                  nargs=2,
                  help="Include optional feature")

parser.parse_args(['--with', 'foo', 'bar'])

with_callback:
	option: <Option at 0x11045fd40: --with>
	opt_str: --with
	value: ('foo', 'bar')
	parser: <optparse.OptionParser instance at 0x11045f638>


(<Values at 0x11045fab8: {'with': None}>, [])

### Step 2: Download the JSON Data

In [116]:
%%writefile quickWeather.py

import json
import requests
import optparse
from pprint import pprint


def main():
    parser = optparse.OptionParser(usage='%prog -l <location>', version='0.1')
    parser.add_option('-l',
                      dest='location',
                      type='string',
                      help='To insert location')

    (options, args) = parser.parse_args()
    
    # TODO: Download the JSON data from OpenWeatherMap.org's API.
    url = 'http://api.openweathermap.org/data/2.5/forecast/daily?q={}&cnt=3'.format(options.location)
    response = requests.get(url)
    response.raise_for_status()
    
    pprint(response.text)
    
    
    # TODO: Load JSOn data into a Python variable.
    
if __name__ == '__main__':
    main()

Overwriting quickWeather.py


In [119]:
%run quickWeather.py -l Seoul

u'{"cod":"200","message":0.0391,"city":{"id":1835848,"name":"Seoul","coord":{"lon":126.977829,"lat":37.56826},"country":"KR","population":0,"sys":{"population":0}},"cnt":3,"list":[{"dt":1433646000,"temp":{"day":298.4,"min":287.64,"max":299.52,"night":287.64,"eve":298.1,"morn":298.4},"pressure":1005.13,"humidity":61,"weather":[{"id":800,"main":"Clear","description":"sky is clear","icon":"01d"}],"speed":2.37,"deg":211,"clouds":0},{"dt":1433732400,"temp":{"day":296.6,"min":286.28,"max":296.86,"night":289.85,"eve":295.81,"morn":286.28},"pressure":999.19,"humidity":64,"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}],"speed":1.74,"deg":274,"clouds":56},{"dt":1433818800,"temp":{"day":297.38,"min":285.79,"max":298.95,"night":286.28,"eve":297.91,"morn":285.79},"pressure":997.59,"humidity":64,"weather":[{"id":800,"main":"Clear","description":"sky is clear","icon":"02d"}],"speed":1.81,"deg":264,"clouds":8}]}\n'


### Step 3: Load JSON Data and Print Weather

In [ ]:
%%writefile quickWeather.py

import json
import requests
import optparse
from pprint import pprint


def main():
    parser = optparse.OptionParser(usage='%prog -l <location>', version='0.1')
    parser.add_option('-l',
                      dest='location',
                      type='string',
                      help='To insert location')

    (options, args) = parser.parse_args()
    
    # TODO: Download the JSON data from OpenWeatherMap.org's API.
    url = 'http://api.openweathermap.org/data/2.5/forecast/daily?q={}&cnt=3'.format(options.location)
    response = requests.get(url)
    response.raise_for_status()
    
    # TODO: Load JSON data into a Python variable.
    weatherData = json.loads(response.text)
    pprint(weatherData)
    print('')
    
    # Print weather descriptions.
    w = weatherData['list']
    print('Current weather in {}:'.format(options.location))
    print('{} - {}'.format(w[0]['weather'][0]['main'], 
                         w[0]['weather'][0]['description']))
    print('')
    print('Tomorrow:')
    print('{} - {}'.format(w[1]['weather'][0]['main'], 
                          w[1]['weather'][0]['description']))
    print('')
    print('Day after tomorrow:')
    print('{} - {}'.format(w[2]['weather'][0]['main'],
                   w[2]['weather'][0]['description']))
    
if __name__ == '__main__':
    main()

In [137]:
%%writefile quickWeather.py

import json
import requests
import optparse
from pprint import pprint

def print_weather(w, i):
    print('{} - {}'.format(w[i]['weather'][0]['main'], 
                         w[i]['weather'][0]['description']))
    print('')


def main():
    parser = optparse.OptionParser(usage='%prog -l <location>', version='0.1')
    parser.add_option('-l',
                      dest='location',
                      type='string',
                      help='To insert location')

    (options, args) = parser.parse_args()
    
    # TODO: Download the JSON data from OpenWeatherMap.org's API.
    url = 'http://api.openweathermap.org/data/2.5/forecast/daily?q={}&cnt=3'.format(options.location)
    response = requests.get(url)
    response.raise_for_status()
    
    # TODO: Load JSON data into a Python variable.
    weatherData = json.loads(response.text)
    pprint(weatherData)
    print('')
    
    # Print weather descriptions.
    w = weatherData['list']
    print('Current weather in {}:'.format(options.location))
    
    for i, item in enumerate(w):
        if i == 0:
            print_weather(w, i)
        if i == 1:
            print('Tomorrow:')
            print_weather(w, i)
        if i == 2:
            print('Day after tomorrow:')
            print_weather(w, i)
    
if __name__ == '__main__':
    main()

Overwriting quickWeather.py


In [138]:
%run quickWeather.py -l Seoul

{u'city': {u'coord': {u'lat': 37.56826, u'lon': 126.977829},
           u'country': u'KR',
           u'id': 1835848,
           u'name': u'Seoul',
           u'population': 0,
           u'sys': {u'population': 0}},
 u'cnt': 3,
 u'cod': u'200',
 u'list': [{u'clouds': 0,
            u'deg': 211,
            u'dt': 1433646000,
            u'humidity': 61,
            u'pressure': 1005.13,
            u'speed': 2.37,
            u'temp': {u'day': 298.4,
                      u'eve': 298.1,
                      u'max': 299.52,
                      u'min': 287.64,
                      u'morn': 298.4,
                      u'night': 287.64},
            u'weather': [{u'description': u'sky is clear',
                          u'icon': u'01d',
                          u'id': 800,
                          u'main': u'Clear'}]},
           {u'clouds': 56,
            u'deg': 274,
            u'dt': 1433732400,
            u'humidity': 64,
            u'pressure': 999.19,
            u'speed

### Ideas for Similar Programs

- Collect weather forecasts for several campsites or hiking trails to see which one will have the best weather.
- Schedule a program to regularly check the weather and send you a frost alert if you need to move your plaints indoors. (Chapter 15 covers scheduling, and Chapter 16 explains how to send email.)
- Pull weather data from multiple sites to show all at aonce, or calculate and show the average of the multiple weather predictions.

## Summary

- CSV and JSON는 데이터 저장하는 plaintext다. 인간이 읽기 쉽다.
- 간단한 스프레드시트나 웹 앱 데이터에 많이 사용한다.

## Pratice Project

### Excel-to-CSV Converter

- 엑셀은 여러개의 시트를 포함하고 있다.
- 1개의 시트당 1개의 csv 파일을 만든다.
        <excel filename>_<sheet title>.csv 로 저장

#### 여러 개 파일 동시에 열기

- [How can I open multiple files using "with open" in Python? \- Stack Overflow](http://stackoverflow.com/questions/4617034/how-can-i-open-multiple-files-using-with-open-in-python)
```python
with open('a', 'w') as a, open('b', 'w') as b:
    do_something()
```

In [ ]:
# 전체적인 큰 그림

for excelFile in os.listdir('.'):
    # Skip non-xlsx files, load the workbook object.
    for sheetName in wb.get_sheet_names():
        # Loop through every sheet in the workbook.
        sheet = wb.get_sheet_by_name(sheetName)

        # Create the CSV filename from the Excel filename and sheet title.
        # Create the csv.writer object for this CSV file.

        # Loop through every row in the sheet.
        for rowNum in range(1, sheet.get_highest_row() + 1):
            rowData = []    # append each cell to this list
            # Loop through each cell in the row.
            for colNum in range(1, sheet.get_highest_column() + 1):
                # Append each cell's data to rowData.

            # Write the rowData list to the CSV file.

        csvFile.close()